In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib


In [ ]:
# Check if TensorFlow is using a GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow is using the following GPU(s):")
    for gpu in gpus:
        print(gpu)
else:
    print("No GPU detected for TensorFlow.")


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
# Paths
data_dir = 'data/training'  # Folder with .tif images
csv_file = 'data/training.csv'  # CSV file with image_id and is_homogeneous


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
# Strip any leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Now you can access 'image_id' without the extra space
image_id_values = df['image_id'].values
print(image_id_values)


In [ ]:
def load_and_preprocess_image(image_path):
    # Load image with PIL and convert to an array
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    # Normalize image pixel values (0-255 -> 0-1)
    img_array = img_array / 255.0
    return img_array

In [ ]:
print(df.columns)



In [ ]:
# 3. Create lists of image paths and labels
# Assuming image_id values need to be 3 digits with leading zeros
image_paths = [os.path.join(data_dir, f"{str(image_id).zfill(3)}.tif") for image_id in df['image_id']]
labels = df['is_homogenous'].values

In [ ]:
# 4. Load images and preprocess them
images = np.array([load_and_preprocess_image(image_path) for image_path in image_paths])



In [ ]:
# 5. Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Assuming y_train is a NumPy array of labels
# Separate majority and minority class samples
X_train_majority = X_train[y_train == 0]  # Majority class (e.g., Heterogeneous)
y_train_majority = y_train[y_train == 0]

X_train_minority = X_train[y_train == 1]  # Minority class (e.g., Homogeneous)
y_train_minority = y_train[y_train == 1]

print(f"Majority class samples: {len(y_train_majority)}")
print(f"Minority class samples: {len(y_train_minority)}")


In [ ]:
# Data augmentation parameters for the minority class
minority_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


In [ ]:
# Calculate how many augmented images we need
num_majority_samples = len(y_train_majority)
num_minority_samples = len(y_train_minority)
num_augmented_images_needed = num_majority_samples - num_minority_samples

print(f"Number of augmented images needed: {num_augmented_images_needed}")


In [ ]:
# Reshape X_train_minority for the generator
X_train_minority = X_train_minority.reshape((-1, IMG_SIZE[0], IMG_SIZE[1], 3))

# Create a generator for the minority class
minority_generator = minority_datagen.flow(
    X_train_minority,
    y_train_minority,
    batch_size=1,
    shuffle=False
)

# Generate augmented images
X_augmented = []
y_augmented = []

for i in range(num_augmented_images_needed):
    X_batch, y_batch = next(minority_generator)
    X_augmented.append(X_batch[0])
    y_augmented.append(y_batch[0])

# Convert to NumPy arrays
X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)

print(f"Augmented images generated: {len(y_augmented)}")


In [ ]:
# Combine original minority data with augmented data
X_minority_balanced = np.concatenate([X_train_minority, X_augmented])
y_minority_balanced = np.concatenate([y_train_minority, y_augmented])

# Combine with majority class data
X_train_balanced = np.concatenate([X_train_majority, X_minority_balanced])
y_train_balanced = np.concatenate([y_train_majority, y_minority_balanced])

print(f"Balanced training samples: {len(y_train_balanced)}")


In [ ]:
from sklearn.utils import shuffle

X_train_balanced, y_train_balanced = shuffle(X_train_balanced, y_train_balanced, random_state=42)


In [ ]:
# Data generator for the balanced dataset (minimal augmentation)
train_datagen_balanced = ImageDataGenerator(
    rescale=1./255  # Normalize pixel values
)

train_generator = train_datagen_balanced.flow(
    X_train_balanced, y_train_balanced, batch_size=BATCH_SIZE
)

# Validation data generator (normalize)
val_datagen = ImageDataGenerator(
    rescale=1./255
)

val_generator = val_datagen.flow(
    X_val, y_val, batch_size=BATCH_SIZE
)


In [ ]:
# 6. Create data generators for augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [ ]:
val_datagen = ImageDataGenerator()


In [ ]:
# 7. Create the data generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE)


In [ ]:
# 8. Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# 10. Create the model by adding custom layers on top of the pre-trained base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [ ]:
# 11. Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 12. Train the model
history = model.fit(
    train_generator,
    epochs=30,  # Adjust the number of epochs as needed
    validation_data=val_generator
)

In [ ]:
# 13. Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

In [ ]:
model.save('vgg16_homogeneous_classification.h5')


In [ ]:
from tensorflow.keras.models import load_model


In [ ]:
model1 = load_model('vgg16_homogeneous_classification.h5')  # Path to your saved model


In [ ]:
val_loss, val_accuracy = model1.evaluate(X_val, y_val)


In [ ]:
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

In [ ]:
predictions = model.predict(X_val)
predicted_labels = (predictions >= 0.5).astype(int)  # Threshold at 0.5 to get binary labels


In [ ]:
from sklearn.metrics import classification_report


In [ ]:
print(classification_report(y_val, predicted_labels, target_names=['Heterogeneous', 'Homogeneous']))


In [ ]:

# Assuming y_val contains the true labels and predicted_labels contains the predicted labels

# Step 1: Calculate n_0 and n_1
n_0 = np.sum(y_val == 0)  # Number of true heterogeneous cells
n_1 = np.sum(y_val == 1)  # Number of true homogeneous cells

# Step 2: Calculate a_0 and a_1
a_0 = np.sum((y_val == 0) & (predicted_labels == 0))  # Correctly predicted as heterogeneous
a_1 = np.sum((y_val == 1) & (predicted_labels == 1))  # Correctly predicted as homogeneous

# Step 3: Calculate the score
if n_0 == 0 or n_1 == 0:
    score = 0  # Handle edge cases where there are no samples of a class
else:
    score = (a_0 * a_1) / (n_0 * n_1)

print(f'Score: {score}')
